In [ ]:
!pip install gensim==4.3.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 26.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
import tensorflow
import re
from gensim.models import KeyedVectors 
from keras.preprocessing.text import Tokenizer  
from tensorflow.keras.preprocessing.sequence import pad_sequences


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
2024-03-08 14:04:10.949610: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-08 14:04:11.084038: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dler

In [ ]:
!pip install wordninja

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 24.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541552 sha256=b147ff1036c6043e5acf532c61214b1c1083684d5c60148ed29150fb30ce4f21
  Stored in directory: /root/.cache/pip/wheels/27/58/04/5df38b06c20f0d5f2a631b392c92d7b6fc2e84a335e6446097
Successfully built wordninja

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install pyspellchecker


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 89.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
df = pd.read_csv('10K_rows.csv')

In [ ]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
df.head()

,text,class
0,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,Am I weird I don't get affected by compliments...,non-suicide
2,Finally 2020 is almost over... So I can never ...,non-suicide
3,i need helpjust help me im crying so hard,suicide
4,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [ ]:
#normalizing
df['text'] = df['text'].str.replace("’", "'")

In [ ]:
df["text"] = df["text"].str.lower()

In [ ]:
abb = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "dont": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "idk": "i do not know",
  "he'd've": "he would have",
  "he'll": "he will",  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "i'd": "i would",
  "i'd've": "i would have",
  "i'll": "i will",
  "i'll've": "i will have",
  "i'm": "i am",
  "im": "i am",
  "i've": "i have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is", "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have", "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",      "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have", "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have", "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

abb_re = re.compile('(%s)' % '|'.join(abb.keys()))

def expandContractions(text, abb_re=abb_re):
    def replace(match):
        return abb[match.group(0)]
    return abb_re.sub(replace, text)

df['text'] = df['text'].apply(expandContractions)

In [ ]:
df.head()

,text,class
0,ex wife threatening suiciderecently i left my ...,suicide
1,am i weird i do not get affected by compli ame...,non-suicide
2,finally 2020 is almost over... so i can never ...,non-suicide
3,i need helpjust help me i am crying so hard,suicide
4,"i am so losthello, my name is adam (16) and i ...",suicide


In [ ]:
# Function to tokenize text
def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens

# Apply tokenization to the text column in the DataFrame
df['text'] = df['text'].apply(tokenize_text)

In [ ]:
df.head()

,text,class
0,"[ex, wife, threatening, suiciderecently, i, le...",suicide
1,"[am, i, weird, i, do, not, get, affected, by, ...",non-suicide
2,"[finally, 2020, is, almost, over, ..., so, i, ...",non-suicide
3,"[i, need, helpjust, help, me, i, am, crying, s...",suicide
4,"[i, am, so, losthello, ,, my, name, is, adam, ...",suicide


In [ ]:
import wordninja
import re

# Apply word segmentation to the 'text' column in the DataFrame
df['text'] = df['text'].apply(lambda tokens: wordninja.split(" ".join(tokens)))

In [ ]:
df.head(10)

,text,class
0,"[ex, wife, threatening, suicide, recently, i, ...",suicide
1,"[am, i, weird, i, do, not, get, affected, by, ...",non-suicide
2,"[finally, 2020, is, almost, over, so, i, can, ...",non-suicide
3,"[i, need, help, just, help, me, i, am, crying,...",suicide
4,"[i, am, so, lost, hello, my, name, is, adam, 1...",suicide
5,"[hone, t, ly, i, do, not, know, i, do, not, kn...",suicide
6,"[trigger, warning, excuse, for, self, inflicte...",suicide
7,"[it, ends, tonight, i, can, not, do, it, anymo...",suicide
8,"[everyone, wants, to, be, edgy, ', ', and, it,...",non-suicide
9,"[my, life, is, over, at, 20, years, old, hello...",suicide


In [ ]:
# Function to remove stopwords from a list of tokens
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    tokens_without_stopwords = [
        token for token in tokens if token.lower() not in stop_words]
    return tokens_without_stopwords

# Function to remove punctuation from a list of tokens
def remove_punctuation(tokens):
    tokens_without_punctuation = [token for token in tokens if token.isalnum()]
    return tokens_without_punctuation

# Function to remove numbers from a list of tokens
def remove_numbers(tokens):
    tokens_without_numbers = [token for token in tokens if not token.isdigit()]
    return tokens_without_numbers

# Apply the functions to the text column in the DataFrame
df['text'] = df['text'].apply(remove_stopwords)
df['text'] = df['text'].apply(remove_punctuation)
df['text'] = df['text'].apply(remove_numbers)


In [ ]:
df.head(10)

,text,class
0,"[ex, wife, threatening, suicide, recently, lef...",suicide
1,"[weird, get, affected, comp, li, ment, coming,...",non-suicide
2,"[finally, almost, never, hear, bad, year, ever...",non-suicide
3,"[need, help, help, crying, hard]",suicide
4,"[lost, hello, name, adam, struggling, years, a...",suicide
5,"[hone, ly, know, know, even, feel, like, nothi...",suicide
6,"[trigger, warning, excuse, self, inflicted, bu...",suicide
7,"[ends, tonight, anymore, quit]",suicide
8,"[everyone, wants, edgy, making, self, consciou...",non-suicide
9,"[life, years, old, hello, year, old, balding, ...",suicide


In [ ]:
# Function to remove extra whitespaces from each word in a list
def remove_whitespace(tokens):
    cleaned_tokens = [token.strip() for token in tokens]
    return cleaned_tokens

# Function to remove symbols and digits from each word in a list
def remove_symbols_digits(tokens):
    cleaned_tokens = [re.sub('[^a-zA-Z\s]', ' ', token) for token in tokens]
    return cleaned_tokens

# Function to remove special characters from each word in a list
def remove_special(tokens):
    cleaned_tokens = [token.replace("\r", " ").replace("\n", " ") for token in tokens]
    return cleaned_tokens

# Apply other preprocessing functions
df['text'] = df['text'].apply(remove_whitespace)
df['text'] = df['text'].apply(remove_symbols_digits)
df['text'] = df['text'].apply(remove_special)

In [ ]:
df.head(10)

,text,class
0,"[ex, wife, threatening, suicide, recently, lef...",suicide
1,"[weird, get, affected, comp, li, ment, coming,...",non-suicide
2,"[finally, almost, never, hear, bad, year, ever...",non-suicide
3,"[need, help, help, crying, hard]",suicide
4,"[lost, hello, name, adam, struggling, years, a...",suicide
5,"[hone, ly, know, know, even, feel, like, nothi...",suicide
6,"[trigger, warning, excuse, self, inflicted, bu...",suicide
7,"[ends, tonight, anymore, quit]",suicide
8,"[everyone, wants, edgy, making, self, consciou...",non-suicide
9,"[life, years, old, hello, year, old, balding, ...",suicide


In [ ]:
# Function to remove URLs from a list of strings
def remove_url(tokens):
    cleaned_tokens = [re.sub(r'http\S+', '', token) for token in tokens]
    return cleaned_tokens

# Function to remove email addresses from a list of strings
def remove_mail(tokens):
    cleaned_tokens = [re.sub(r'\S+@\S+', '', token) for token in tokens]
    return cleaned_tokens

# Function to remove emojis from a list of strings
def remove_emoji(tokens):
    cleaned_tokens = [re.sub(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U0001FB00-\U0001FBFF\U0001FE00-\U0001FE0F\U0001F004]+', '', token) for token in tokens]
    return cleaned_tokens

# Apply the functions to the 'text' column
df['text'] = df['text'].apply(remove_url)
df['text'] = df['text'].apply(remove_mail)
df['text'] = df['text'].apply(remove_emoji)

In [ ]:
df.head(10)

,text,class
0,"[ex, wife, threatening, suicide, recently, lef...",suicide
1,"[weird, get, affected, comp, li, ment, coming,...",non-suicide
2,"[finally, almost, never, hear, bad, year, ever...",non-suicide
3,"[need, help, help, crying, hard]",suicide
4,"[lost, hello, name, adam, struggling, years, a...",suicide
5,"[hone, ly, know, know, even, feel, like, nothi...",suicide
6,"[trigger, warning, excuse, self, inflicted, bu...",suicide
7,"[ends, tonight, anymore, quit]",suicide
8,"[everyone, wants, edgy, making, self, consciou...",non-suicide
9,"[life, years, old, hello, year, old, balding, ...",suicide


In [ ]:
#remove words with only 2 characters
#df['text'] = df['text'].apply(lambda x: [word for word in x if len(word) > 2])

In [ ]:
df.head(10)

,text,class
0,"[ex, wife, threatening, suicide, recently, lef...",suicide
1,"[weird, get, affected, comp, li, ment, coming,...",non-suicide
2,"[finally, almost, never, hear, bad, year, ever...",non-suicide
3,"[need, help, help, crying, hard]",suicide
4,"[lost, hello, name, adam, struggling, years, a...",suicide
5,"[hone, ly, know, know, even, feel, like, nothi...",suicide
6,"[trigger, warning, excuse, self, inflicted, bu...",suicide
7,"[ends, tonight, anymore, quit]",suicide
8,"[everyone, wants, edgy, making, self, consciou...",non-suicide
9,"[life, years, old, hello, year, old, balding, ...",suicide


In [ ]:
!python -m spacy download en_core_web_sm

2024-03-08 14:05:07.218068: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-08 14:05:07.377159: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-08 14:05:07.377198: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-03-08 14:05:07.414397: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-08 14:05:08.351709: W tensorflow/stream_executor/pla

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

# Define a function to correct a word
def correct_word(text):
    # Join the tokens back into a sentence
    text_with_space = " ".join(text)
    # Process the text with spaCy
    doc = nlp(text_with_space)
    # Get the corrected tokens
    corrected_tokens = [token.text_with_ws for token in doc]
    return corrected_tokens

# Apply correction to each row in the "text" column
df["text"] = df["text"].apply(correct_word)


/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-08 14:05:18.510933: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-03-08 14:05:18.510987: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2024-03-08 14:05:18.511012: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-cc725fbb-272a-46d8-8511-cf7b3f199438): /proc/driver/nvidia/version does not exist


In [ ]:
df.head()

,text,class
0,"[ex , wife , threatening , suicide , recently ...",suicide
1,"[weird , get , affected , comp , li , ment , c...",non-suicide
2,"[finally , almost , never , hear , bad , year ...",non-suicide
3,"[need , help , help , crying , hard]",suicide
4,"[lost , hello , name , adam , struggling , yea...",suicide


In [ ]:
def remove_all_spaces(data):

  cleaned_text = [[re.sub(r'\s+', '', token) for token in sentence] for sentence in data]  # Apply to tokens
  return cleaned_text

df['text'] = remove_all_spaces(df['text'])

In [ ]:
df.head()

,text,class
0,"[ex, wife, threatening, suicide, recently, lef...",suicide
1,"[weird, get, affected, comp, li, ment, coming,...",non-suicide
2,"[finally, almost, never, hear, bad, year, ever...",non-suicide
3,"[need, help, help, crying, hard]",suicide
4,"[lost, hello, name, adam, struggling, years, a...",suicide


In [ ]:
#POS Tagging
# Load the English model
nlp = spacy.load("en_core_web_sm")

# Function to perform POS tagging
def pos_tag(text):
    # Create tuples of (word, POS tag) for each token
    pos_tuples = [(token.text, token.pos_) for token in nlp(" ".join(text))]
    # Return the list of tuples
    return pos_tuples

# Apply POS tagging using a new column
df['pos_tags'] = df['text'].apply(pos_tag)

In [ ]:
df.head()

,text,class,pos_tags
0,"[ex, wife, threatening, suicide, recently, lef...",suicide,"[(ex, PROPN), (wife, NOUN), (threatening, VERB..."
1,"[weird, get, affected, comp, li, ment, coming,...",non-suicide,"[(weird, ADJ), (get, AUX), (affected, VERB), (..."
2,"[finally, almost, never, hear, bad, year, ever...",non-suicide,"[(finally, ADV), (almost, ADV), (never, ADV), ..."
3,"[need, help, help, crying, hard]",suicide,"[(need, VERB), (help, NOUN), (help, VERB), (cr..."
4,"[lost, hello, name, adam, struggling, years, a...",suicide,"[(lost, VERB), (hello, INTJ), (name, PROPN), (..."


In [ ]:
#Lemmatization
# Load the spaCy English model
nlp = spacy.load('en_core_web_sm')

def lemmatize_tokens(tokens):
    # Join the tokens back into a sentence
    text = ' '.join(tokens)
    # Process the text using spaCy
    doc = nlp(text)
    # Lemmatize each token and return the lemmatized tokens
    lemmatized_tokens = [token.lemma_ for token in doc]
    return lemmatized_tokens

# Apply lemmatization to the tokenized text column in the DataFrame
df['text'] = df['text'].apply(lemmatize_tokens)

In [ ]:
df.head(10)

,text,class,pos_tags
0,"[ex, wife, threaten, suicide, recently, leave,...",suicide,"[(ex, PROPN), (wife, NOUN), (threatening, VERB..."
1,"[weird, get, affect, comp, li, ment, come, som...",non-suicide,"[(weird, ADJ), (get, AUX), (affected, VERB), (..."
2,"[finally, almost, never, hear, bad, year, ever...",non-suicide,"[(finally, ADV), (almost, ADV), (never, ADV), ..."
3,"[need, help, help, cry, hard]",suicide,"[(need, VERB), (help, NOUN), (help, VERB), (cr..."
4,"[lose, hello, name, adam, struggle, year, afra...",suicide,"[(lost, VERB), (hello, INTJ), (name, PROPN), (..."
5,"[hone, ly, know, know, even, feel, like, nothi...",suicide,"[(hone, PROPN), (ly, X), (know, VERB), (know, ..."
6,"[trigger, warn, excuse, self, inflict, burn, k...",suicide,"[(trigger, NOUN), (warning, VERB), (excuse, NO..."
7,"[end, tonight, anymore, quit]",suicide,"[(ends, VERB), (tonight, NOUN), (anymore, ADV)..."
8,"[everyone, want, edgy, make, self, conscious, ...",non-suicide,"[(everyone, PRON), (wants, VERB), (edgy, ADV),..."
9,"[life, year, old, hello, year, old, balding, m...",suicide,"[(life, NOUN), (years, NOUN), (old, ADJ), (hel..."


In [ ]:
def combine_tokens(tokens):
  sentence = " ".join(tokens)
  return sentence

df['text'] = df['text'].apply(combine_tokens)

In [ ]:
# Rename the column
df = df.rename(columns={'text': 'lemmatized_text'})

In [ ]:
df.head()

,lemmatized_text,class,pos_tags
0,ex wife threaten suicide recently leave wife g...,suicide,"[(ex, PROPN), (wife, NOUN), (threatening, VERB..."
1,weird get affect comp li ment come someone kno...,non-suicide,"[(weird, ADJ), (get, AUX), (affected, VERB), (..."
2,finally almost never hear bad year ever swear ...,non-suicide,"[(finally, ADV), (almost, ADV), (never, ADV), ..."
3,need help help cry hard,suicide,"[(need, VERB), (help, NOUN), (help, VERB), (cr..."
4,lose hello name adam struggle year afraid past...,suicide,"[(lost, VERB), (hello, INTJ), (name, PROPN), (..."


In [ ]:
df.to_csv("cleaned_suicide_data.csv", index=True)

In [ ]:
from keras.utils import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the pre-trained Word2Vec model by Google
import gensim.downloader as api
from gensim.models import Word2Vec

# Download a pre-trained model
model_name = "word2vec-google-news-300"
word2vec_model = api.load(model_name)

# Tokenize and pad text
texts = df['lemmatized_text']
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

sequences = tokenizer.texts_to_sequences(texts)
maxLen = 150
sequences = pad_sequences(sequences, maxlen=maxLen)

# Create an embedding matrix
vocab_len = len(tokenizer.word_index) + 1
embed_vector_len = 300  
embedding_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in tokenizer.word_index.items():
    if word in word2vec_model:
        embedding_vector = word2vec_model[word]
        embedding_matrix[index] = embedding_vector

# Create a feature matrix with average word embeddings
feature_matrix = np.array([
    np.mean([embedding_matrix[word_idx] for word_idx in seq if word_idx != 0], axis=0)
    for seq in sequences
])

df['word2vec_model'] = feature_matrix

[=================================================-] 99.7% 1657.6/1662.8MB downloaded


KernelInterrupted: Execution interrupted by the Jupyter kernel.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cc725fbb-272a-46d8-8511-cf7b3f199438' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>